In [17]:
import os

import pandas as pd
import numpy as np
from neo4j import Transaction, Session, GraphDatabase

from graph_copy import graph_copy
from neo4jexplorer import Neo4jExplorer

In [18]:
HIST_URI = "neo4j+s://99c1a702.databases.neo4j.io:7687"
LOCAL_URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PSWD = "231099"
hist_driver = GraphDatabase.driver(HIST_URI, auth=(USER, PSWD))
local_driver = GraphDatabase.driver(LOCAL_URI, auth=(USER, "23109900"))

In [19]:
from one_GESN_graph_creation import main

# main()
graph_copy(hist_driver.session(), local_driver.session())

      n_name      n_din
0  6.68-30-1  6.68-30-1
1  6.68-31-1  6.68-31-1
2  6.68-30-2  6.68-30-2
3  6.68-31-2  6.68-31-2
4  6.68-30-3  6.68-30-3
       n_din      m_din  weight
0  6.68-30-1  6.68-31-1       1
1  6.68-31-1  6.68-13-1       1
2  6.68-31-1  6.68-33-1       1
3  6.68-30-2  6.68-31-2       1
4  6.68-31-2  6.68-33-1       1


In [20]:
app = Neo4jExplorer()#uri=LOCAL_URI)
print(len(app.get_all_dins()))
app.del_loops()
hist_gesns = app.get_all_dins()
len(hist_gesns)

133
4x loops deleted


118

In [21]:
df1 = pd.read_excel("./../new_data/Сметы Берзарина.xls")
df1.rename(
    columns={"Проект": "wbs1", "Смета": "wbs2", "Шифр": "wbs3_id", "Наименование": "name"},
    inplace=True
)
df1 = df1[["wbs1", "wbs2", "wbs3_id", "name"]]
df1.dropna(subset=["wbs3_id"], inplace=True)
df1.drop_duplicates(inplace=True)
df1.shape

(720, 4)

In [22]:
input_gesns = df1.wbs3_id.unique()
targ_gesns = np.intersect1d(hist_gesns, input_gesns)
len(targ_gesns)

56

In [23]:
"3.47-3-15" in targ_gesns

True

In [24]:
app.create_new_graph_algo(targ_gesns)

4x loops deleted
deleted: 15.1-2201-03
deleted: 15.1-2201-04
deleted: 15.1-2300-02
deleted: 15.1-2500-01
deleted: 15.2-41-7
deleted: 3.47-34-8
deleted: 6.68-30-5
deleted: 6.68-31-5
deleted: 15.1-2102-02
deleted: 3.47-9-10
deleted: 3.47-11-1
deleted: 3.47-9-5
deleted: 3.8-1-1
deleted: 3.22-49-1
deleted: 3.4-6-10
deleted: 3.29-1587-2
deleted: 3.29-1588-1
deleted: 3.29-1590-2
deleted: 3.29-1591-1
deleted: 3.4-10-2
deleted: 6.69-8-1
deleted: 3.16-9-2
deleted: 3.16-9-3
deleted: 3.23-13-1
deleted: 6.69-8-2
deleted: 3.16-15-2
deleted: 3.7-45-1
deleted: 3.24-2-1
deleted: 3.24-2-3
deleted: 3.8-2-6
deleted: 3.27-54-1
deleted: 3.7-46-3
deleted: 3.22-71-1
deleted: 3.24-36-2
deleted: 3.24-38-1
deleted: 3.24-40-1
deleted: 3.24-40-2
deleted: 3.24-41-1
deleted: 3.24-41-2
deleted: 3.24-42-1
deleted: 3.24-42-2
deleted: 3.24-43-1
deleted: 3.4-19-36
deleted: 3.24-43-2
deleted: 3.24-44-1
deleted: 3.24-44-2
deleted: 3.4-8-53
deleted: 3.23-1-5
deleted: 3.26-27-1
deleted: 4.11-1-4
deleted: 4.11-3-1
deleted: 4

In [27]:
len(app.get_all_dins())

56

In [26]:
app.driver.session().run(
"""
match (n)-[r]->(m)
where n.DIN = '3.47-7-8' AND m.DIN = '3.47-3-15'
delete r
"""
)